<a href="https://cocl.us/Data_Science_with_Scalla_top"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0103EN/adds/Data_Science_with_Scalla_notebook_top.png" width = 750, align = "center"></a>
 <br/>
<a><img src="https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width="200" align="center"></a>"

# Module 2: Preparing Data 

## Identifying Outliers

### Lesson Objectives

After completing this lesson, you should be able to:

- Compute the inverse of covariance matrix given of a dataset
-	Compute Mahalanobis Distance for all elements in a dataset
-	Remove outliers from a dataset


## Mahalanobis Distance 

-	Multi-dimensional generalization of measuring how many standard deviations a point is away from the mean
-	Measured along each Principal Component axis 
-	Unitless and scale-invariant 
-	Takes into account the correlations of the dataset
-	Used to detect outliers

In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
import spark.implicits._

import org.apache.spark.sql.functions._
import  org.apache.spark.mllib.linalg.{Vector, Vectors}
import  org.apache.spark.ml.feature.StandardScaler
import  org.apache.spark.ml.feature.VectorAssembler 
import org.apache.spark.mllib.util.MLUtils


spark = org.apache.spark.sql.SparkSession@27c3e8bf


org.apache.spark.sql.SparkSession@27c3e8bf

In [2]:
// Continuing from Previous Example 

val  dfRandom = spark.range(0, 10).select("id").
 withColumn("uniform", rand(10L)).
 withColumn("normal1", randn(10L)).
 withColumn("normal2", randn(11L))

val  assembler = new VectorAssembler().
 setInputCols(Array("uniform","normal1","normal2")).
 setOutputCol("features")

val dfVec = MLUtils.convertVectorColumnsFromML(assembler.transform(dfRandom))

dfRandom = [id: bigint, uniform: double ... 2 more fields]
assembler = vecAssembler_9ba67429882b
dfVec = [id: bigint, uniform: double ... 3 more fields]


[id: bigint, uniform: double ... 3 more fields]

In [3]:
// Continuing from the Previous Example
dfVec.select("id","features").show()

// An Example with Outliers 
val dfOutlier = dfVec.select("id","features").unionAll(spark.createDataFrame(Seq((10,Vectors.dense(3, 3, 3)))))
dfOutlier.sort(dfOutlier("id").desc).show(5)

+---+--------------------+
| id|            features|
+---+--------------------+
|  0|[0.41371264720975...|
|  1|[0.73117192818966...|
|  2|[0.19829196382083...|
|  3|[0.12714181165849...|
|  4|[0.76043181534066...|
|  5|[0.12030715258495...|
|  6|[0.12131363910425...|
|  7|[0.44292918521277...|
|  8|[0.88987842538862...|
|  9|[0.03650707717266...|
+---+--------------------+

+---+--------------------+
| id|            features|
+---+--------------------+
| 10|       [3.0,3.0,3.0]|
|  9|[0.03650707717266...|
|  8|[0.88987842538862...|
|  7|[0.44292918521277...|
|  6|[0.12131363910425...|
+---+--------------------+
only showing top 5 rows



dfOutlier = [id: bigint, features: vector]


[id: bigint, features: vector]

In [4]:
// An Example with Outliers 

val scaler = new StandardScaler().
 setInputCol("features").setOutputCol("scaledFeat").
 setWithStd(true).setWithMean(true)

val scalerModel = scaler.fit(MLUtils.convertVectorColumnsToML(dfOutlier.select("id","features")))

val dfScaled = scalerModel.transform(MLUtils.convertVectorColumnsToML(dfOutlier)).select("id","scaledFeat")
dfScaled.sort(dfScaled("id").desc).show(3)

+---+--------------------+
| id|          scaledFeat|
+---+--------------------+
| 10|[2.82164986866882...|
|  9|[-0.6946827334846...|
|  8|[0.31788506121507...|
+---+--------------------+
only showing top 3 rows



scaler = stdScal_cfcd508615c2
scalerModel = stdScal_cfcd508615c2
dfScaled = [id: bigint, scaledFeat: vector]


[id: bigint, scaledFeat: vector]

In [5]:
import  org.apache.spark.mllib.stat.Statistics

import  breeze.linalg._

val  rddVec = MLUtils.convertVectorColumnsFromML(dfScaled.select("scaledFeat")).rdd.map(_(0).asInstanceOf[org.apache.spark.mllib.linalg.Vector])

val  colCov = Statistics.corr(rddVec)
val  invColCovB = inv(new DenseMatrix(3, 3, colCov.toArray))

rddVec = MapPartitionsRDD[39] at map at <console>:44
colCov = 
invColCovB = 


1.0                 0.8103496428958891  0.7305190864811277
0.8103496428958891  1.0                 0.718094148661543
0.7305190864811277  0.718094148661543   1.0
3.358697500406212    -1.9816807132192142  -1.0305593050850421
-1.9816807132192142  3.2338825769455832   -0.8745765716449652
-1.030559305085042   -0.8745765716449652  2.380871560770073


In [ ]:
// Computing Mahalanobis Distance 

val mahalanobis = udf[Double, org.apache.spark.ml.linalg.Vector]{ v =>
 val k = v.toArray
 val vB = new DenseVector(k);
 vB.t * invColCovB * vB
}

val dfMahalanobis = dfScaled.withColumn("mahalanobis", mahalanobis(dfScaled("scaledFeat")))

In [ ]:
// Computing Mahalanobis Distance 
dfMahalanobis.show()

In [ ]:
// Removing Outliers 

dfMahalanobis.sort(dfMahalanobis("mahalanobis").desc).show(2)

val ids = dfMahalanobis.select("id","mahalanobis").sort(dfMahalanobis("mahalanobis").desc).drop("mahalanobis").collect() 

val idOutliers = ids.map(_(0).asInstanceOf[Long]).slice(0,2)

## Removing Outliers

In [ ]:
dfOutlier.filter("id not in (10, 2)").show()

## Lesson Summary

- Having completed this lesson, you should be able to:
-	Compute the inverse of covariance matrix given of a dataset 
-	Compute Mahalanobis Distance for all elements in a dataset
-	Remove outliers from a dataset

### About the Authors

[Petro Verkhogliad](https://www.linkedin.com/in/vpetro) is Consulting Manager at Lightbend. He holds a Masters degree in Computer Science with specialization in Intelligent Systems. He is passionate about functional programming and applications of AI.